In [1]:
CODE_DIR = '/home/shreyas/Documents/git/wheelai/'
traindata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/sample_train/'
validdata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/valid/'
results_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/results/'

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Lambda, Cropping2D, Activation

%matplotlib inline

Using TensorFlow backend.


### Create helper fucntions

In [3]:
batch_size = 64
data_dim = (160, 320)

For further reading on [keras Imagegenerator](https://keras.io/preprocessing/image/)

In [4]:
def get_batches(path, class_mode='categorical', gen=image.ImageDataGenerator(), \
                shuffle=True, target_size=data_dim, batch_size=1):
    '''
    Args
    path: path to data directory
    calss_mode: 'categorical', 'binary', 'sparse'
    gen: keras image generator
    shuffle: if to shuffle data or not
    target_size: out dimensions of the image
    Yields
    batch of given dimension
    '''
    return gen.flow_from_directory(path, class_mode=class_mode, batch_size=batch_size, \
                                   target_size=target_size, shuffle=shuffle)

def get_steps(batches, batch_size):
    '''Return number of times the batches to train on for keras fit_generator'''
    steps = int(batches.samples/batch_size)
    return (steps if batches.samples%batch_size==0 else (steps+1))


In [6]:
# get train and valid batches
train_b = get_batches(traindata_path, batch_size=batch_size)
valid_b = get_batches(validdata_path, batch_size=batch_size)

Found 12954 images belonging to 3 classes.
Found 3375 images belonging to 3 classes.


In [7]:

train_steps = get_steps(train_b, batch_size)
valid_steps = get_steps(valid_b, batch_size)
num_class = train_b.num_class

In [8]:
# get class labels
train_labels = train_b.classes
valid_labels = valid_b.classes

In [9]:
# one hot encode the labels
y_train = keras.utils.to_categorical(train_labels)
y_valid = keras.utils.to_categorical(valid_labels)

In [10]:
print(train_labels.shape, valid_labels.shape, y_train.shape, y_valid.shape)

(12954,) (3375,) (12954, 3) (3375, 3)


In [11]:
# generate batches to calculating the conv features
# NOTE: CLASS_MODE = NONE because you we are running batches for prediction and not training
trn_b = get_batches(traindata_path, class_mode=None, shuffle=False, batch_size=batch_size)
val_b = get_batches(validdata_path, class_mode=None, shuffle=False, batch_size=batch_size)

Found 12954 images belonging to 3 classes.
Found 3375 images belonging to 3 classes.


In [12]:
# Data Augmentation
gen_augdata = image.ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, 
                                      shear_range=0.2, zoom_range=0.2)
augtrain_b = get_batches(traindata_path, gen=gen_augdata, class_mode='categorical',\
                         batch_size=batch_size)

Found 12954 images belonging to 3 classes.


### Define models </br>
Read more about [VGG with Keras](https://keras.io/applications/)

In [13]:
def vgg_conv():
    '''Returns a VGG Model without last Conv Block'''
    
    # load the model with pretained weights from keras
    model = VGG16(include_top=False, weights='imagenet', input_shape=(90,320,3))
    # get the index for last conv layer
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Conv2D][-1]
    # get rid of last conv block
    conv_layers = layers[:15]
    # create a VGG model
    conv_model = Sequential(conv_layers)
    # add preprocess layer on top of VGG model
    model = Sequential()
    # Cropp the image to just keep the road, making it easy for our neural net
    model.add(Cropping2D(cropping=((70,0), (0,0)), input_shape=(160, 320, 3)))
    # Normalise the data to 0
    model.add(Lambda(lambda x: (x / 255.0) - 0.5)) 
    model.add(conv_model)
    return model

def conv_layer(input_shape):
    '''Add top for VGG based only on Conv layers'''
    model = Sequential()
    model.add(BatchNormalization(input_shape=input_shape))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    # Maxpool with 1,2 to make match rows and coloumns
    model.add(MaxPooling2D((1,2)))
    # number of filters/channels is equal to number of labels
    # in this case 3
    model.add(Conv2D(3, (3,3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    return model
    
def top_layer(input_shape):
    '''Add custom top layer to VGG'''
    model = Sequential()
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(256, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_class, activation='softmax'))
    
    return model 

In [14]:
def get_ft(trn_b, train_steps, val_b, valid_steps):
    '''Calculate the feature for VGG model'''
    model = vgg_conv()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # get the output VGG last conv layer
    trn_ft = model.predict_generator(trn_b, train_steps)
    val_ft = model.predict_generator(val_b, valid_steps)
    print(trn_ft.shape, val_ft.shape)
    # save for further use
    np.save(results_path + 'trn_ft.npy', trn_ft)
    np.save(results_path + 'val_ft.npy', val_ft)

### Get VGG features

In [15]:
# calcualte the features for VGG
get_ft(trn_b, train_steps, val_b, valid_steps)

(12954, 5, 20, 512) (3375, 5, 20, 512)


In [16]:
# load the output of VGG
X_train = np.load(results_path + 'trn_ft.npy')
X_valid = np.load(results_path + 'val_ft.npy')

### Train top layer, and save bottlenect weights

In [17]:
# load VGG top layer to calculate bottlenect weights
model = top_layer(X_train.shape[1:])

In [18]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=2e-5), metrics=['accuracy'])

In [19]:
filepath1 = results_path+'vgg_bottleneck.h5'
# define the checkpoint
#checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks1=[checkpoint1]

In [20]:
model.fit(X_train, y_train, epochs=21, batch_size=32, verbose=1, validation_data=(X_valid, y_valid))
model.save_weights(filepath1)

Train on 12954 samples, validate on 3375 samples
Epoch 1/21
12954/12954 [==============================] - 21s - loss: 1.1966 - acc: 0.5728 - val_loss: 0.6905 - val_acc: 0.7061
Epoch 2/21
12954/12954 [==============================] - 20s - loss: 0.8817 - acc: 0.6789 - val_loss: 0.6700 - val_acc: 0.7185
Epoch 3/21
12954/12954 [==============================] - 20s - loss: 0.7702 - acc: 0.7138 - val_loss: 0.7315 - val_acc: 0.6960
Epoch 4/21
12954/12954 [==============================] - 20s - loss: 0.6765 - acc: 0.7414 - val_loss: 0.7234 - val_acc: 0.6975
Epoch 5/21
12954/12954 [==============================] - 21s - loss: 0.6342 - acc: 0.7551 - val_loss: 0.7663 - val_acc: 0.6794
Epoch 6/21
12954/12954 [==============================] - 20s - loss: 0.5612 - acc: 0.7869 - val_loss: 0.7321 - val_acc: 0.7108
Epoch 7/21
12954/12954 [==============================] - 20s - loss: 0.5150 - acc: 0.8028 - val_loss: 0.7752 - val_acc: 0.6889
Epoch 8/21
12954/12954 [==============================]

### Build and finetune VGG

In [21]:
def vgg_model():
    '''Build complete VGG model'''
    base_model = vgg_conv()
    # freeze first 11 layers
    for layer in base_model.layers[:11]: layer.trainable=False
    print (base_model.output_shape[1:])
    # create and load top layer
    top_model = top_layer(base_model.output_shape[1:])
    top_model.load_weights(results_path+'vgg_bottleneck.h5')
    # join VGG with top layer
    base_model.add(top_model)
    
    return base_model

In [22]:
vgg_model = vgg_model()
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
vgg_model.compile(optimizer=adam,loss='categorical_crossentropy')

(5, 20, 512)


In [23]:
#filepath2 = results_path+'vgg_ft.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1,\
#                              save_best_only=False, mode='min', period=1)
#callbacks2=[checkpoint2]

In [24]:
vgg_model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2,
                        validation_data=valid_b, validation_steps=valid_steps)
model.save_weights(results_path+'vgg_ft1.h5')

Epoch 1/2
203/203 [==============================] - 235s - loss: 1.4652 - val_loss: 0.8182
Epoch 2/2
203/203 [==============================] - 222s - loss: 1.0997 - val_loss: 0.9582


In [25]:
model.optimizer.lr = 1e-5
vgg_model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=3,
                        validation_data=valid_b, validation_steps=valid_steps)
model.save_weights(results_path+'vgg_ft2.h5')

Epoch 1/3
203/203 [==============================] - 237s - loss: 1.0078 - val_loss: 0.8358
Epoch 2/3
203/203 [==============================] - 219s - loss: 0.9357 - val_loss: 0.7976
Epoch 3/3
203/203 [==============================] - 225s - loss: 0.8878 - val_loss: 0.7986


In [26]:
model.optimizer.lr = 1e-3
vgg_model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=2,
                        validation_data=valid_b, validation_steps=valid_steps)
model.save_weights(results_path+'vgg_ft3.h5')

Epoch 1/2
203/203 [==============================] - 232s - loss: 0.8848 - val_loss: 0.7750
Epoch 2/2
203/203 [==============================] - 221s - loss: 0.8259 - val_loss: 0.8495


In [27]:
model.optimizer.lr = 1e-4
vgg_model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=5,
                        validation_data=valid_b, validation_steps=valid_steps)
model.save_weights(results_path+'vgg_ft4.h5')

Epoch 1/5
203/203 [==============================] - 232s - loss: 0.8012 - val_loss: 0.7727
Epoch 2/5
203/203 [==============================] - 226s - loss: 0.7566 - val_loss: 0.8443
Epoch 3/5
203/203 [==============================] - 221s - loss: 0.7518 - val_loss: 0.7678
Epoch 4/5
203/203 [==============================] - 220s - loss: 0.7401 - val_loss: 0.8264
Epoch 5/5
203/203 [==============================] - 225s - loss: 0.7222 - val_loss: 0.8638


In [28]:
model.optimizer.lr = 1e-3
vgg_model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=2,
                        validation_data=valid_b, validation_steps=valid_steps)
model.save_weights(results_path+'vgg_ft5.h5')

Epoch 1/2
203/203 [==============================] - 175s - loss: 0.4800 - val_loss: 0.8046
Epoch 2/2
203/203 [==============================] - 169s - loss: 0.3701 - val_loss: 0.9397


In [29]:
model.optimizer.lr = 1e-5
vgg_model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=3,
                        validation_data=valid_b, validation_steps=valid_steps)
model.save_weights(results_path+'vgg_ft6.h5')

Epoch 1/3
203/203 [==============================] - 177s - loss: 0.2836 - val_loss: 0.8786
Epoch 2/3
203/203 [==============================] - 169s - loss: 0.2364 - val_loss: 1.0083
Epoch 3/3
203/203 [==============================] - 169s - loss: 0.1771 - val_loss: 1.0878


In [30]:
# serialize model to JSON
model_path = results_path+'vgg_ft.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

Saved model to disk
